<a href="https://colab.research.google.com/github/AtulJoshi1/Optimal-Retail-Store-Placement/blob/master/Retail_Geo_Spotting_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math
from sklearn.neighbors import DistanceMetric

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_nyc_checkins = pd.read_csv('/content/drive/My Drive/BDA/Datasets/Foursquare/DataSets/NYC TOKyo checkin/dataset_TSMC2014_NYC.tsv',sep='\t',encoding='ISO-8859-1',names=['User_ID','Venue_ID','Venue_category_ID','Venue_category_name','Latitude','Longitude','Timezone_offset','UTC_Time'],header=None)

In [4]:
df_nyc_checkins.head()

,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012


In [0]:
df_nyc_checkins['Latitude'][0]

40.71981037548853

In [5]:
df_nyc_checkins.shape

(227428, 8)

### Data

In [0]:
x=df_nyc_checkins.iloc[:,3:6]

In [0]:
x

,Venue_category_name,Latitude,Longitude
0,Arts & Crafts Store,40.719810,-74.002581
1,Bridge,40.606800,-74.044170
2,Home (private),40.716162,-73.883070
3,Medical Center,40.745164,-73.982519
4,Food Truck,40.740104,-73.989658
...,...,...,...
227423,Music Venue,40.733596,-74.003139
227424,Burger Joint,40.745719,-73.993720
227425,Home (private),40.854364,-73.883070
227426,Bar,40.735981,-74.029309


In [0]:
count_locations=df_nyc_checkins.groupby(['Latitude','Longitude']).size().reset_index().rename(columns={0:'count'})

In [0]:
count_locations

,Latitude,Longitude,count
0,40.550852,-73.931293,6
1,40.556602,-73.913372,2
2,40.557295,-73.911548,13
3,40.558871,-73.923459,1
4,40.559423,-73.926507,1
...,...,...,...
42976,40.983151,-74.010437,1
42977,40.983611,-74.015588,1
42978,40.983991,-73.978946,3
42979,40.987646,-73.988880,16


In [0]:
print("No. of unique locations: ",count_locations.shape[0])

No. of unique locations:  42981


In [0]:
count_locations_category=x.groupby(['Venue_category_name']).size().reset_index().rename(columns={0:'count'})

In [0]:
count_locations_category

,Venue_category_name,count
0,Afghan Restaurant,4
1,African Restaurant,40
2,Airport,2852
3,American Restaurant,3701
4,Animal Shelter,62
...,...,...
246,Video Store,42
247,Vietnamese Restaurant,180
248,Winery,31
249,Wings Joint,149


## Working on Reduced data

In [0]:
df_checkin = df_nyc_checkins.iloc[:10000,:]

In [0]:
df_checkin['UTC_Time']=pd.to_datetime(df_checkin['UTC_Time'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_checkin['index']=(df_checkin.index.to_list())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_checkin

,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time,index
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00,0
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00,1
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00,3
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00,4
...,...,...,...,...,...,...,...,...,...
9995,389,4b37853af964a520c54125e3,4bf58dd8d48988d1fa931735,Hotel,40.775186,-73.963302,-240,2012-04-11 17:06:11+00:00,9995
9996,663,4baa673bf964a520d8663ae3,4bf58dd8d48988d124941735,Office,40.773502,-73.981026,-240,2012-04-11 17:08:18+00:00,9996
9997,217,4d93828762ad5481a4cd834b,4bf58dd8d48988d124941735,Office,40.755661,-73.975389,-240,2012-04-11 17:08:38+00:00,9997
9998,1066,4ae0e652f964a520818321e3,4bf58dd8d48988d1ed941735,Spa / Massage,40.766741,-73.982156,-240,2012-04-11 17:09:12+00:00,9998


In [0]:
count_df_locations =df_checkin.groupby(['Latitude','Longitude']).size().reset_index().rename(columns={0:'count'})

In [0]:
count_df_locations

,Venue_category_name,Latitude,Longitude,count
0,African Restaurant,40.684159,-73.954289,1
1,African Restaurant,40.684198,-73.722027,7
2,African Restaurant,40.686415,-73.954635,1
3,African Restaurant,40.689566,-73.971935,2
4,Airport,40.639681,-73.979935,1
...,...,...,...,...
5742,Wings Joint,40.737980,-74.030761,1
5743,Wings Joint,40.765263,-73.981582,1
5744,Wings Joint,40.815624,-74.162222,2
5745,Wings Joint,40.848644,-74.080977,1


In [0]:
count_df_locations =df_checkin.groupby(['Venue_category_name','Latitude','Longitude']).size().reset_index().rename(columns={0:'count'})

In [0]:
count_df_locations['Latitude']

0       40.684159
1       40.684198
2       40.686415
3       40.689566
4       40.639681
          ...    
5742    40.737980
5743    40.765263
5744    40.815624
5745    40.848644
5746    40.767444
Name: Latitude, Length: 5747, dtype: float64

In [0]:
print("No. of unique locations: ",len(count_df_locations))

No. of unique locations:  5747


In [0]:
count_df_locations['Latitude'][0]

40.684159398078926

In [0]:
len(count_df_locations['Venue_category_name'].unique())

212

In [0]:
count_df_locations.iloc[:,1:-1]

,Latitude,Longitude
0,40.684159,-73.954289
1,40.684198,-73.722027
2,40.686415,-73.954635
3,40.689566,-73.971935
4,40.639681,-73.979935
...,...,...
5742,40.737980,-74.030761
5743,40.765263,-73.981582
5744,40.815624,-74.162222
5745,40.848644,-74.080977


In [0]:
count_df_locations.head()

,Venue_category_name,Latitude,Longitude,count
0,African Restaurant,40.684159,-73.954289,1
1,African Restaurant,40.684198,-73.722027,7
2,African Restaurant,40.686415,-73.954635,1
3,African Restaurant,40.689566,-73.971935,2
4,Airport,40.639681,-73.979935,1


In [0]:
dist_mat = DistanceMetric.get_metric('haversine').pairwise(count_df_locations.iloc[:,1:-1] )*6371

In [0]:
dist_mat.shape

(5747, 5747)

In [0]:
print(dist_mat)

[[   0.         1461.56774789   14.53249036 ... 1561.85487356
  1320.80852852  542.22531486]
 [1461.56774789    0.         1464.07399819 ... 2913.57781798
  2507.25587896 1667.71039199]
 [  14.53249036 1464.07399819    0.         ... 1552.47912199
  1308.14713069  527.71794715]
 ...
 [1561.85487356 2913.57781798 1552.47912199 ...    0.
   558.69220448 1249.03966609]
 [1320.80852852 2507.25587896 1308.14713069 ...  558.69220448
     0.          865.71596628]
 [ 542.22531486 1667.71039199  527.71794715 ... 1249.03966609
   865.71596628    0.        ]]


In [0]:
count_df_locations['Venue_category_name'][0]

'African Restaurant'

In [0]:
lst = [[] for i in range(dist_mat.shape[0])]
for i in range(dist_mat.shape[0]):
  for j in range(dist_mat.shape[1]):
    if dist_mat[i][j]<=100:
      lst[i].append((count_df_locations['Latitude'][j],count_df_locations['Longitude'][j]))
  #dic.update(count_locations[i],lst)

In [0]:
len(lst)

5747

In [0]:
lst[0]

[(40.684159398078926, -73.954289),
 (40.686414660176055, -73.95463511518253),
 (40.671638993520205, -73.96381202180841),
 (40.699108, -73.955248),
 (40.67241900279851, -73.96293496396677),
 (40.67682902605353, -73.96343804293126),
 (40.68193081222049, -73.94995122562862),
 (40.683419, -73.96637199999999),
 (40.68721847389229, -73.95717791452198),
 (40.672307292956866, -73.95784374357963),
 (40.68183918319128, -73.96677481467539),
 (40.687944, -73.96260874000002),
 (40.688462, -73.96271),
 (40.689876, -73.95139),
 (40.67228503425415, -73.9627870309687),
 (40.677492, -73.963759),
 (40.67986739997525, -73.96423214181858),
 (40.689522232043856, -73.95129203796388),
 (40.69084011789372, -73.95571231842041),
 (40.677823, -73.968826),
 (40.68470716795056, -73.94382640962874),
 (40.69032712271465, -73.95148383433914),
 (40.68919682411797, -73.95734310150145),
 (40.680630900000004, -73.94967198371887),
 (40.687415187557306, -73.96922528743744),
 (40.67843772387125, -73.9444406220427),
 (40.6799

# Extracting Features

## 1. Density

In [0]:
density = [len(lst[i]) for i in range(0,len(lst))]

In [0]:
len(density)

5747

In [0]:
density[5]

23

In [0]:
count_df_locations

,Venue_category_name,Latitude,Longitude,count
0,African Restaurant,40.684159,-73.954289,1
1,African Restaurant,40.684198,-73.722027,7
2,African Restaurant,40.686415,-73.954635,1
3,African Restaurant,40.689566,-73.971935,2
4,Airport,40.639681,-73.979935,1
...,...,...,...,...
5742,Wings Joint,40.737980,-74.030761,1
5743,Wings Joint,40.765263,-73.981582,1
5744,Wings Joint,40.815624,-74.162222,2
5745,Wings Joint,40.848644,-74.080977,1


In [0]:
count_df_locations[count_df_locations['Latitude']==lst[0][0][0]].Venue_category_name=='African Restaurant'

0    True
Name: Venue_category_name, dtype: bool

## 2. Neighbors' Entropy and 3. Competitiveness

In [0]:
categories = count_df_locations['Venue_category_name'].unique()

In [0]:
len(categories)

212

In [0]:
#lst_cat=[[] for i in range(len(lst))]
dic={}
for i in range(len(lst)):
  new_lst=[]
  for j in range(len(lst[i])):
    new_lst.append((j,count_df_locations['Venue_category_name'][j]))
  dic[(i,count_df_locations['Venue_category_name'][i])] = new_lst

In [0]:
cat_i=np.array(dic[(0, 'African Restaurant')])[:,1:2]

In [0]:
categ = np.unique(cat_i,return_counts=True)[0]

In [0]:
categ

array(['African Restaurant', 'Airport', 'American Restaurant'],
      dtype='<U21')

In [0]:
neigh_entropy=[]
for i in list(dic.keys()):
  fraction_class=[]
  entropy=0
  cat_i=np.array(dic[i])[:,1:2]
  total_count = len(cat_i)
  fraction_class=np.unique(cat_i,return_counts=True)[1]
  fraction_class = fraction_class/np.sum(fraction_class)
  for j in fraction_class:
          entropy=entropy-j*math.log2(j)
  neigh_entropy.append(entropy)

In [0]:
neigh_entropy[0]

NameError: ignored

In [0]:
np.unique(cat_i,return_counts=True)[0][0] in dic[i][1]

True

In [0]:
np.unique(cat_i,return_counts=True)

(array(['African Restaurant', 'Airport', 'American Restaurant',
        'Animal Shelter', 'Antique Shop', 'Arcade', 'Art Gallery',
        'Art Museum', 'Arts & Crafts Store', 'Arts & Entertainment',
        'Asian Restaurant', 'Athletic & Sport', 'Automotive Shop',
        'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Bar'], dtype='<U21'),
 array([  4,  30, 118,   2,   1,   9,  14,  10,   7,   1,  25,  16,  17,
         32,  14,  51,  71, 131]))

In [0]:
(0, 'African Restaurant')[1]

'African Restaurant'

In [0]:
def entropy_competi(dic):
  neigh_entropy=[]
  competitiveness=[]
  for i in list(dic.keys()):
    fraction_class=[]
    competition=0
    entropy=0
    cat_i=np.array(dic[i])[:,1:2]
    total_count = len(cat_i)
    categ = np.unique(cat_i,return_counts=True)[0]
    fraction_class=np.unique(cat_i,return_counts=True)[1]
    fraction_class = fraction_class/np.sum(fraction_class)
    for j in range(len(categ)):
      if(categ[j] == i[1]):
        competition = fraction_class[j]
        break
    competitiveness.append(competition)
    for k in fraction_class:
            entropy=entropy-k*math.log2(k)
    neigh_entropy.append(entropy)
  return((competitiveness,neigh_entropy))

In [0]:
comp,ne = entropy_competi(dic)

In [0]:
ne[0]

1.2832547532764527

In [0]:
len(neigh_entropy)

5747

In [0]:
neigh_entropy[0]

1.2832547532764527

In [0]:
np.unique(cat_i,return_counts=True)[0]

array(['African Restaurant', 'Airport', 'American Restaurant',
       'Animal Shelter', 'Antique Shop', 'Arcade', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store', 'Arts & Entertainment',
       'Asian Restaurant', 'Athletic & Sport', 'Automotive Shop',
       'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Bar'], dtype='<U21')

In [0]:
len(neigh_entropy)

5747

In [0]:
len(density)

5747

## 4. Quality by Jensen

In [0]:
categories[0]

'African Restaurant'

In [0]:
np.unique(np.array(dic[list(dic.keys())[0]])[:,1:2])

array(['African Restaurant', 'Airport', 'American Restaurant'],
      dtype='<U21')

In [0]:
neig_mat = np.zeros((len(categories),len(categories)))

In [0]:

neig_mat.shape

(212, 212)

In [0]:
neig_mat[0][0]

0.0

In [0]:
np.unique(np.array(dic[(0, 'African Restaurant')])[:,1:2],return_counts=True)[1]

In [0]:
def count_cat(j,i):
  if(dic[j][1]==i): 
    count+=i


In [0]:
for j in range(len(lst)):
  for i in categories:
    if(dic[j[1]==i]):
      count_i+=1 
    

## 5. Area Popularity

In [0]:
count_df_locations.head()

,Venue_category_name,Latitude,Longitude,count
0,African Restaurant,40.684159,-73.954289,1
1,African Restaurant,40.684198,-73.722027,7
2,African Restaurant,40.686415,-73.954635,1
3,African Restaurant,40.689566,-73.971935,2
4,Airport,40.639681,-73.979935,1


In [0]:
np.sum([density[i]==0 for i in range(len(density))])

0

In [0]:
lst[4]

[(40.63968085124788, -73.97993533649185),
 (40.63546678381779, -73.99186684082225),
 (40.6365, -73.98679),
 (40.636735711488974, -73.99040342059492),
 (40.634856, -73.965126),
 (40.63949763333333, -73.97983020000002),
 (40.63605511596764, -73.99116669606713),
 (40.653046, -73.973021),
 (40.62586036296825, -73.97625071067725),
 (40.648436832434776, -73.97850626102887),
 (40.651622570231545, -73.97572268998049),
 (40.63981002719504, -73.96745208982402),
 (40.639901, -73.983202),
 (40.6394869711505, -73.97977360650297),
 (40.639413333333344, -73.97987413333334),
 (40.642719, -73.97949),
 (40.643109, -73.983272),
 (40.626571000000006, -73.97696897971106),
 (40.639413333333344, -73.97982253333333),
 (40.634242310367384, -73.98054369471026),
 (40.64715352960956, -73.97881044263973),
 (40.63063339189206, -73.97712595918333),
 (40.632300871701176, -73.98887973002674),
 (40.64053341403471, -73.97712595918333),
 (40.642200982303464, -73.98887973002674),
 (40.639489815065495, -73.97976889853065),

In [0]:
count=0
area_pop=[]
for j in lst[4]:
    for k in range(count_df_locations.shape[1]):
      if(j[0]==count_df_locations['Latitude'].values[k]):
        print(count)
        count+=count_df_locations['count'][k]
area_pop.append(count)

In [0]:
area_pop

[0]

In [0]:
count_df_locations['count'][0]

1

In [0]:
lst[4][0][0]==40.63968085124788

True

In [0]:
lst[4][0][0] in count_df_locations['Latitude'].values

True

In [0]:
lst[0][0][0]

40.684159398078926

In [0]:
count_df_locations['Latitude'][0]

40.684159398078926

In [0]:
area_pop=[]
for i in range(len(lst)):
  count=0
  for j in lst[i]:
    for k in range(count_df_locations.shape[1]):
      if(j[0]==count_df_locations['Latitude'][k]):
        print((j[0],k))
        count+=count_df_locations['count'][k]
  area_pop.append(count)

(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68419809161396, 1)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.684159398078926, 0)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.686414660176055, 2)
(40.684159398078926, 0)
(40.68956576688808, 3)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.68956576688808, 3)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576688808, 3)
(40.68419809161396, 1)
(40.68956576688808, 3)
(40.686414660176055, 2)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.684159398078926, 0)
(40.686414660176055, 2)
(40.68956576

In [0]:
np.sum(area_pop)

536

In [0]:
area_pop[4]

0

## 6. Transition Density

In [0]:
df_checkin.head()

,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time,index
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00,0
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00,1
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00,3
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00,4


In [0]:
df_checkin.iloc[:,4:6]

,Latitude,Longitude
0,40.719810,-74.002581
1,40.606800,-74.044170
2,40.716162,-73.883070
3,40.745164,-73.982519
4,40.740104,-73.989658
...,...,...
9995,40.775186,-73.963302
9996,40.773502,-73.981026
9997,40.755661,-73.975389
9998,40.766741,-73.982156


In [0]:
x=[tuple(i) for i in count_df_locations.iloc[:,1:-1].values]

In [0]:
df_checkin['Location']=[tuple(i) for i in df_checkin.iloc[:,4:6].values]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_checkin

,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time,index,Location
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00,0,"(40.71981037548853, -74.00258103213994)"
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00,1,"(40.606799581406435, -74.04416981025437)"
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2,"(40.71616168484322, -73.88307005845945)"
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00,3,"(40.7451638, -73.982518775)"
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00,4,"(40.74010382743943, -73.98965835571289)"
...,...,...,...,...,...,...,...,...,...,...
9995,389,4b37853af964a520c54125e3,4bf58dd8d48988d1fa931735,Hotel,40.775186,-73.963302,-240,2012-04-11 17:06:11+00:00,9995,"(40.77518644745349, -73.96330162882805)"
9996,663,4baa673bf964a520d8663ae3,4bf58dd8d48988d124941735,Office,40.773502,-73.981026,-240,2012-04-11 17:08:18+00:00,9996,"(40.77350198619314, -73.98102578853477)"
9997,217,4d93828762ad5481a4cd834b,4bf58dd8d48988d124941735,Office,40.755661,-73.975389,-240,2012-04-11 17:08:38+00:00,9997,"(40.75566091379096, -73.97538900375365)"
9998,1066,4ae0e652f964a520818321e3,4bf58dd8d48988d1ed941735,Spa / Massage,40.766741,-73.982156,-240,2012-04-11 17:09:12+00:00,9998,"(40.76674068362153, -73.98215640859391)"


In [0]:
df_checkin[df_checkin['User_ID']==87]

,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time,index,Location
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00,4,"(40.74010382743943, -73.98965835571289)"
10,87,4d8263a73e916dcb8edd80d2,4bf58dd8d48988d155941735,Gastropub,40.743483,-73.994009,-240,2012-04-03 18:07:15+00:00,10,"(40.74348254, -73.99400899999999)"
3001,87,40b68100f964a5207d001fe3,4bf58dd8d48988d163941735,Park,40.742188,-73.987924,-240,2012-04-06 14:45:49+00:00,3001,"(40.74218830848454, -73.98792418909154)"
3058,87,4e6122c0e4cdf1e2bebadc68,4bf58dd8d48988d124941735,Office,40.754117,-73.979874,-240,2012-04-06 15:32:59+00:00,3058,"(40.754116747135726, -73.97987365722656)"
6523,87,4edcd3198231efe39240ba6f,4bf58dd8d48988d124941735,Office,40.749262,-73.983981,-240,2012-04-09 14:16:55+00:00,6523,"(40.74926221909231, -73.9839814563788)"
6708,87,4f27112ae4b0d10db310bf14,4bf58dd8d48988d107951735,Clothing Store,40.738185,-73.992182,-240,2012-04-09 16:24:45+00:00,6708,"(40.738185367290185, -73.99218247901545)"
8103,87,4ac915d4f964a52085be20e3,4bf58dd8d48988d179941735,Bagel Shop,40.756126,-73.964462,-240,2012-04-10 10:42:03+00:00,8103,"(40.756125558705904, -73.96446172990993)"
8108,87,4b5200d2f964a520946027e3,4bf58dd8d48988d16d941735,Café,40.756216,-73.964284,-240,2012-04-10 10:47:36+00:00,8108,"(40.75621625245672, -73.96428379916298)"
8711,87,49f1e5bff964a520c4691fe3,4bf58dd8d48988d123941735,Bar,40.739386,-73.991850,-240,2012-04-10 22:22:38+00:00,8711,"(40.739385999999996, -73.99185)"
9370,87,3fd66200f964a52003e81ee3,4bf58dd8d48988d1d5941735,Hotel,40.756119,-73.972532,-240,2012-04-11 03:58:05+00:00,9370,"(40.75611913826422, -73.97253157753975)"


In [0]:
df_checkin['User_ID'].unique()

array([ 470,  979,   69,  395,   87,  484,  642,  292,  428,  877,  625,
        691,  116,  931,   53,  445,  135,  895,  704,  891,  284,  901,
        803,  699,  100,  443,  689,  390,   32,  230,  834,  962, 1032,
        525,  318, 1047,  738,  364,  398,  839,  424,  751,  889,  612,
        643,  718,  768,   15,  458,  280, 1016,  281,  366,  276,  227,
         80,  673,  337,  916,  639,  752,  713,  449, 1012,  749,  438,
        217,  656,  225,  628,  504,  226,  164,  235,  118, 1054,  881,
        706,  143, 1005,  912,  537,  406,  868, 1055,  907,  303,  793,
        784,  734,  540,  816,  623,  950,  294,  615,  385,  173,  801,
         16,  467,    7,  592,  669,  574,  436,  636,  865,  621,  272,
       1063,  591,  976,  562,  783,  319,  468,  874,  107,  667,  557,
        725,  774,  880,  288,  335,  769,  194,  403,  573,  346,  968,
        539,  184,  144,   71,  688,   65,  885,  267,  820,  516,  176,
        528,   30,  246,  515,  710,  145,  498,  8

In [0]:
lst[0][0]

(40.684159398078926, -73.954289)

In [0]:
df_checkin['Location']

0        (40.71981037548853, -74.00258103213994)
1       (40.606799581406435, -74.04416981025437)
2        (40.71616168484322, -73.88307005845945)
3                    (40.7451638, -73.982518775)
4        (40.74010382743943, -73.98965835571289)
                          ...                   
9995     (40.77518644745349, -73.96330162882805)
9996     (40.77350198619314, -73.98102578853477)
9997     (40.75566091379096, -73.97538900375365)
9998     (40.76674068362153, -73.98215640859391)
9999                     (40.754174, -73.971846)
Name: Location, Length: 10000, dtype: object

In [0]:
temp_df

,level_0,User_ID,Venue_ID,Venue_category_ID,Venue_category_name,Latitude,Longitude,Timezone_offset,UTC_Time,index,Location


In [0]:
def get_count(temp_df):
  count=0
  for i in range(len(temp_df)):
    for j in range(1,len(temp_df)):
      if(temp_df.Location[i]!=temp_df.Location[j] and (0<=(temp_df['UTC_Time'][j]- temp_df['UTC_Time'][i]).days<=1)):
        #print(i,j)
        count+=1
        print("count:",count)
  return(count)